In [ ]:
from sklearn.utils import resample

def transform_X(X):
    if isinstance(X, (pd.DataFrame, pd.Series)):
        X = X.values
    if len(X.shape) == 1:
        X = X.reshape(-1, 1)
    return X

def treatment_effect(reg, X, treat_var=0, control_val=0, treat_val=1):
    X = transform_X(X)
    X_control, X_treat = X.copy(), X.copy()
    X_control[:,treat_var] = control_val
    X_treat[:,treat_var] = treat_val
    return reg.predict(X_treat) - reg.predict(X_control)
    
def ate(reg, X, treat_var=0, control_val=0, treat_val=1):
    X = transform_X(X)
    X_control, X_treat = X.copy(), X.copy()
    X_control[:,treat_var] = control_val
    X_treat[:,treat_var] = treat_val
    return (reg.predict(X_treat) - reg.predict(X_control)).mean()

def bootstrap_ate(reg, X, y, treat_var=0, control_val=0, treat_val=1, bootstrap_samples=1000, stratify=None):
    def sample_ate():
        X_sample, y_sample = resample(X, y, stratify=stratify)
        reg.fit(X_sample, y_sample)
        return ate(reg, X_sample, treat_var, control_val, treat_val)

    X = transform_X(X)
    return np.array([sample_ate() for _ in range(bootstrap_samples)])

In [ ]:
from sklearn.linear_model import LinearRegression

ate_samples = bootstrap_ate(LinearRegression(), treat, y)
print(cross_val_score(LinearRegression(), treat.values.reshape(-1, 1), y).mean())
reg = LinearRegression().fit(treat.values.reshape(-1, 1), y)
print(ate(reg, treat))
print(ate_samples.mean(), ate_samples.std())
print(np.quantile(ate_samples, .025), np.quantile(ate_samples, .975))
print([np.quantile(ate_samples, q) for q in (.05, .25, .5, .75, .95)])